In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install sentencepiece
!pip install huggingface_hub
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.3 MB/s eta 0:00:00


In [2]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained (model)
pipeline = transformers.pipeline(
"text-generation",
model=model, tokenizer=tokenizer, torch_dtype=torch.bfloat16,
trust_remote_code=True, device_map="auto", max_new_tokens=1000, do_sample=True, top_k=10, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [3]:
llm= HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

<ipython-input-3-b79127b6d935>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm= HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})


In [20]:
import os
import torch
import pandas as pd
from langchain import PromptTemplate
from langchain.chains import LLMChain  # Ensure you have this import

# Set environment variable for PyTorch CUDA memory management
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Define your prompt template
template = """
Summarize the below given news article:

{article}
"""
prompt = PromptTemplate(template=template, input_variables=["article"])

# Create the LLMChain with the defined prompt and model
llm_chain = LLMChain(prompt=prompt, llm=llm)  # Ensure 'llm' is defined previously

# Load articles from a CSV file
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/random_selected_summary.csv')  # Replace with your CSV file path

# List to store summaries
summaries = []

# Loop through each article and generate summaries
for index, row in df.iterrows():
    article = row['cleaned_article']  # Adjust column name as necessary

    try:
        # Use autocast for mixed precision if CUDA is available
        device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
        with torch.amp.autocast(device_type=device_type):
            summary = llm_chain.invoke(article)  # Use invoke instead of run
            summaries.append(summary)
            print(f"Processed article {summary}")
    except Exception as e:
        print(f"Error processing article {index}: {e}")
        summaries.append(None)  # Append None or a default message for error handling

    torch.cuda.empty_cache()  # Clear unused memory after each iteration

Processed article {'article': 'Poland abortion Polish women look for help in Germany after terminations all but banned Euronews English EnglishFranaisDeutschItalianoEspaolPortugusTrkeMagyarShqipRomnSrpskiLoaderSearchEventsNewslettersLog InMy Account My Europe World Business EU Policy Green Next Health Travel Culture Video More Programmes1Live My europe Brussels, My Love? Europe Decoded Europe News Smart Regions State Of The Union The Cube Uncovering Europe Unreported Europe Sport Football Now Euronews Debates Euronews Witness No Comment Qatar 365 Spotlight The Global Conversation Top News Stories Today World News Business Line Business Planet Focus Global Japan My Wildest Prediction Real Economy Start Me Up Target The Big Question The Dialogue The Exchange Dub.ai Euronews Tech Talks Hacker Hunter Rethink Sci-Tech Space Smart Health Adventures Conscious Travel Explore Golf Travel Tales Notes From The Usa Soul Of The South Taste Women Beyond Borders Climate Now Ocean Ocean Calls The Road

KeyboardInterrupt: 